# Getting started with Spark and Iceberg

In [ ]:
## Update with your principal user credentials (from Polaris Catalog)

clientId="f5fb404daaa0dfe2"
clientSecret="a06f1727e964d80e4daf4c12af146af0"

In [ ]:
## Start the Spark application and connect to our Polaris Catalog

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('iceberg_lab') \
.config('spark.sql.defaultCatalog', 'polaris') \
.config('spark.sql.catalog.polaris', 'org.apache.iceberg.spark.SparkCatalog') \
.config('spark.sql.catalog.polaris.type', 'rest') \
.config('spark.sql.catalog.polaris.header.X-Iceberg-Access-Delegation','true') \
.config('spark.sql.catalog.polaris.client.region','us-east-1') \
.config('spark.sql.catalog.polaris.uri','http://polaris-catalog:8181/api/catalog') \
.config('spark.sql.catalog.polaris.credential',clientId+':'+clientSecret) \
.config('spark.sql.catalog.polaris.warehouse','polariscatalog') \
.config('spark.sql.catalog.polaris.scope','PRINCIPAL_ROLE:ALL') \
.config('spark.sql.catalog.polaris.token-refresh-enabled', 'true') \
.getOrCreate()

Check that the catalog you created during the Polaris Catalog setup is ready
If your catalog is not showing up, go back and make sure the Polaris Catalog setup is completed and permissions are assigned

In [ ]:
%%sql

SHOW CATALOGS

In [ ]:
%%sql

-- Create a default database.
-- This can be anything, but remember to reference it in the following cells

CREATE DATABASE demo

In [ ]:
%%sql

-- Create an Iceberg table with a few columns

CREATE TABLE demo.users (
    id INT, 
    name STRING
) USING ICEBERG

In [ ]:
%%sql

-- Insert some values into your new table

INSERT INTO demo.users VALUES
  (1, 'roy'),
  (2, 'ori'),
  (3, 'john'),
  (4, 'jason'),
  (5, 'david')

In [ ]:
%%sql

SELECT * FROM demo.users

# Iceberg tables under the hood

In the following sections we'll look at the structure of an Iceberg table.

## Snapshots
When you query the snapshots information table you'll be able to see the current and previous snapshots of your table.
Pay attention to the `summary` column, note that `added-records` equal the number of rows we inserted in the previous statement.

In [ ]:
%%sql

-- Inspect the snapshots table

SELECT * FROM polaris.demo.users.snapshots

### Inserting a new value
Insert a new value to the table. A new snapshot is created and the row is added into a new data file

In [ ]:
%%sql

INSERT INTO demo.users VALUES (6, 'bob');

A second snapshot was created representing the new row we added above.
Pay attention under `summary` column to `added-records` which shows 1 and `total-records` which shows 6

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.snapshots

### Updating existing values
To update an individual value in a table, use the `UPDATE` keyword with an appropriate `WHERE` to identify the row you want to update

In [ ]:
%%sql

UPDATE demo.users
SET name = 'dave'
WHERE id = 5

Inspect the snapshots table and note the new snapshot that was created represents an `overwrite` operation. In this operation, Iceberg deleted a row, `deleted-record=1` which was the row we updated containing the original values. And added a row `added-record=1` which is the row with the new values.

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.snapshots

Inspecting the `manifests` table is another way to understand the changes performed on a specific Iceberg table.

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.manifests

### Deleting values

You can delete values from a table using the `DELETE` keyword and a `WHERE` clause to identify which rows to delete

%%sql

DELETE FROM demo.users
WHERE id = 5

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.snapshots

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.manifests

Notice that when you exectue the following query, no results are returned. This means that no delete files where created when rows were updated or deleted.
Kind of strage no?  Well in fact that's because the table is by default configured to Copy On Write which merges the delete files on write.

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.all_delete_files

### Working with MoR and CoW tables

MoR tables require the query engine to merge data and delete files on read. You can periodically compact these files to make the query engine's life easier.
CoW tables merges the data and delete files when the rows are written. It requires more IO on write, but far less on read.

Spark allows you to configure MoR or CoW for either `delete`, `update` or `merge` operations. This gives you flexibility to control how your tables should be updated.

%%sql

ALTER TABLE demo.users SET TBLPROPERTIES (
  'write.update.mode'='merge-on-read'
)

In the previous statement we changed the mode at which Spark updates rows, using MoR.

Next, lets update a row and see how our table reacts

In [ ]:
%%sql

UPDATE demo.users
SET name='bobby'
WHERE id = 6

Check to see if any delete files were created.  Remember, previously no delete files where created.

In [ ]:
%%sql

SELECT * FROM polaris.demo.users.all_delete_files

Inspecting the `manifests` table also shows that Iceberg created a specific manifest file to track the delete file, along with manifests to track the data files.
You can tell by looking at the `content` column. `0` means manifest tracking data files and `1` means manifest tracking delete files

%%sql

SELECT * FROM polaris.demo.users.manifests